## Hands-on Tutorial: Quantifying and Reducing Gender Stereotypes in Word Embeddings

Ensuring fairness in algorithmically-driven decision-making is important to avoid inadvertent cases of bias and perpetuation of harmful stereotypes. However, modern natural language processing techniques, which learn model parameters based on data, might rely on implicit biases presented in the data to make undesirable stereotypical associations. Such a danger is facing us with word embedding, a popular framework to represent text data as vectors which has been used in many machine learning and natural language processing tasks. Recent results ([1](https://arxiv.org/abs/1607.06520), [2](https://arxiv.org/abs/1608.07187)) show that even word embeddings trained on Google News articles exhibit female/male gender stereotypes to a disturbing extent. This raises concerns because of their widespread use, as we describe, often tends to amplify these biases. 

In the following, we provide step-by-step instructions to demonstrate and quanitfy the biases in word embedding.



In [4]:
# Setup:
# Clone the code repository from https://github.com/tolga-b/debiaswe.git
# mkdir debiaswe_tutorial
# cd debiaswe_tutorial
# git clone https://github.com/tolga-b/debiaswe.git

# To reduce the time of downloading data, we provide as subset of GoogleNews-vectors in the following location:
# https://drive.google.com/file/d/1NH6jcrg8SXbnhpIXRIXF_-KUE7wGxGaG/view?usp=sharing

# For full embeddings:
# Download embeddings at https://github.com/tolga-b/debiaswe and put them on the following directory
# embeddings/GoogleNews-vectors-negative300-hard-debiased.bin
# embeddings/GoogleNews-vectors-negative300.bin

In [5]:
from __future__ import print_function, division
%matplotlib inline
from matplotlib import pyplot as plt
import json
import random
import numpy as np

import debiaswe as dwe
import debiaswe.we as we
from debiaswe.we import WordEmbedding
from debiaswe.data import load_professions

## Part 1: Gender Bias in Word Embedding


### Step 1: Load data
We first load the word embedding trained on a corpus of Google News texts consisting of 3 million English words and terms. The embedding maps each word into a 300-dimension vector. 

In [6]:
# load google news word2vec
E = WordEmbedding('./embeddings/w2v_gnews_small.txt')

# load professions
professions = load_professions()
profession_words = [p[0] for p in professions]

*** Reading data from ./embeddings/w2v_gnews_small.txt
(26423, 300)
26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
Loaded professions
Format:
word,
definitional female -1.0 -> definitional male 1.0
stereotypical female -1.0 -> stereotypical male 1.0


### Step 2: Define gender direction

We define gender direction by the direciton of she - he because they are frequent and do not have fewer alternative word senses (e.g., man can also refer to mankind). In the paper, we discuss alternative approach for defining gender direction (e.g., using PCA).

In [7]:
# gender direction
v_gender = E.diff('she', 'he')

### Step 3: Generating analogies of "Man: x :: Woman : y"

We show that the word embedding model generates gender-streotypical analogy pairs. 
To generate the analogy pairs, we use the analogy score defined in our paper. This score finds word pairs that are well aligned with gender direction as well as within a short distance from each other to preserve topic consistency. 


In [8]:
# analogies gender
a_gender = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender:
    print(a+"-"+b)

Computing neighbors
Mean: 10.219732808538016
Median: 7.0
she-he
herself-himself
her-his
woman-man
daughter-son
businesswoman-businessman
girl-boy
actress-actor
chairwoman-chairman
heroine-hero
mother-father
spokeswoman-spokesman
sister-brother
girls-boys
sisters-brothers
queen-king
niece-nephew
councilwoman-councilman
motherhood-fatherhood
women-men
petite-lanky
ovarian_cancer-prostate_cancer
Anne-John
schoolgirl-schoolboy
granddaughter-grandson
aunt-uncle
matriarch-patriarch
twin_sister-twin_brother
mom-dad
lesbian-gay
husband-younger_brother
gal-dude
lady-gentleman
sorority-fraternity
mothers-fathers
grandmother-grandfather
blouse-shirt
soprano-baritone
queens-kings
Jill-Greg
daughters-sons
grandma-grandpa
volleyball-football
diva-superstar
mommy-kid
Sarah-Matthew
hairdresser-barber
softball-baseball
goddess-god
Aisha-Jamal
waitress-waiter
princess-prince
filly-colt
mare-gelding
ladies-gentlemen
childhood-boyhood
interior_designer-architect
nun-priest
wig-beard
granddaughters-grandso

### Step 4: Analyzing gender bias in word vectors asscoiated with professions

Next, we show that many occupations are unintendedly associated with either male of female by projecting their word vectors onto the gender dimension. 

The script will output the profession words sorted with respect to the projection score in the direction of gender.

In [9]:
# profession analysis gender
sp = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.3116408, 'fashion_designer'),
  (-0.18117762, 'fisherman'),
  (-0.16751873, 'dermatologist'),
  (-0.16570373, 'nanny'),
  (-0.15126215, 'naturalist'),
  (-0.14974959, 'bartender'),
  (-0.14228301, 'financier'),
  (-0.12579738, 'farmer'),
  (-0.12516144, 'screenwriter'),
  (-0.12431554, 'infielder'),
  (-0.1224953, 'environmentalist'),
  (-0.12196568, 'councilor'),
  (-0.120170794, 'priest'),
  (-0.11756935, 'dad'),
  (-0.11756064, 'entertainer'),
  (-0.11224885, 'butler'),
  (-0.10854641, 'footballer'),
  (-0.10638268, 'congressman'),
  (-0.10558808, 'cabbie'),
  (-0.105458714, 'researcher')],
 [(0.07658601, 'maid'),
  (0.078715794, 'biologist'),
  (0.08165435, 'barber'),
  (0.08583453, 'scholar'),
  (0.08647158, 'teenager'),
  (0.08886343, 'curator'),
  (0.09047253, 'parishioner'),
  (0.0912133, 'cinematographer'),
  (0.09179836, 'strategist'),
  (0.09384438, 'jurist'),
  (0.09794031, 'welder'),
  (0.1001438, 'dancer'),
  (0.11356944, 'laborer'),
  (0.1222185, 'firefighter'),
  

## Part 2 Racial Bias

### Step 5: Define racial direction
We define racial direction based on the common names in different Demographic groups. 

In [10]:
names = ["Emily", "Aisha", "Anne", "Keisha", "Jill", "Tamika", "Allison", "Lakisha", "Laurie", "Tanisha", "Sarah",
         "Latoya", "Meredith", "Kenya", "Carrie", "Latonya", "Kristen", "Ebony", "Todd", "Rasheed", "Neil", "Tremayne",
         "Geoffrey", "Kareem", "Brett", "Darnell", "Brendan", "Tyrone", "Greg", "Hakim", "Matthew", "Jamal", "Jay",
         "Leroy", "Brad", "Jermaine"]
names_group1 = [names[2 * i] for i in range(len(names) // 2)]
names_group2 = [names[2 * i + 1] for i in range(len(names) // 2)]

In [11]:
# racial direction
vs = [sum(E.v(w) for w in names) for names in (names_group2, names_group1)]
vs = [v / np.linalg.norm(v) for v in vs]

v_racial = vs[1] - vs[0]
v_racial = v_racial / np.linalg.norm(v_racial)

In [12]:
names_group1

['Emily',
 'Anne',
 'Jill',
 'Allison',
 'Laurie',
 'Sarah',
 'Meredith',
 'Carrie',
 'Kristen',
 'Todd',
 'Neil',
 'Geoffrey',
 'Brett',
 'Brendan',
 'Greg',
 'Matthew',
 'Jay',
 'Brad']

### Step 6: Generating racial biased analogies

Similar to Step 3, we generate analogies that align with the racial dimension. 

In [13]:
# racial analogies
a_racial = E.best_analogies_dist_thresh(v_racial)

for (a,b,c) in a_racial:
    print(a+"-"+b)

asserted-remarked
contends-noting
contended-expressed_concern
asserts-observes
insisted-warned
argued-voiced_concern
emphasized-underscored
believes-predicts
argues-warns
denies-blames
emphasizes-underscores
insisting-vowing
concedes-laments
objected-decried
insists-reckons
emphasize-underscore
instructed-exhorted
prove-proved
contradicted-echoed
adamant-hopeful
stressed-lamented
always-especially
cr-bn
assertion-remark
allege-complained
stated-commented
foreclose-foreclosed
doubted-marveled
asserting-acknowledging
convinced-worried
monies-dollars
solely-mainly
insist-warn
underline-underscoring
clarified-reiterated
replied-chuckled
maintains-boasts
mislead-fooled
affirms-reflects
knew-saw
establishes-creates
amend-amendments
elevates-brings
vehemently_denied-reacted_angrily
argue-complain
ambiguity-uncertainty
negate-outweighed
advocating-decrying
not-too
intend-expect
interfere-hamper
manufacture-manufacturing
defended-applauded
creditor-debt
wants-expects
establish-build
detonate-bo

In [14]:
a_racial

[('asserted', 'remarked', 0.4465047),
 ('contends', 'noting', 0.42904818),
 ('contended', 'expressed_concern', 0.40585706),
 ('asserts', 'observes', 0.40066034),
 ('insisted', 'warned', 0.39154112),
 ('argued', 'voiced_concern', 0.37990075),
 ('emphasized', 'underscored', 0.37033975),
 ('believes', 'predicts', 0.3535955),
 ('argues', 'warns', 0.35310158),
 ('denies', 'blames', 0.34485143),
 ('emphasizes', 'underscores', 0.33518958),
 ('insisting', 'vowing', 0.33100656),
 ('concedes', 'laments', 0.33037168),
 ('objected', 'decried', 0.3284281),
 ('insists', 'reckons', 0.3236775),
 ('emphasize', 'underscore', 0.3143053),
 ('instructed', 'exhorted', 0.30382645),
 ('prove', 'proved', 0.30129677),
 ('contradicted', 'echoed', 0.29830486),
 ('adamant', 'hopeful', 0.2957421),
 ('stressed', 'lamented', 0.29471698),
 ('always', 'especially', 0.29280835),
 ('cr', 'bn', 0.29238242),
 ('assertion', 'remark', 0.28915775),
 ('allege', 'complained', 0.28890195),
 ('stated', 'commented', 0.28725165),
 

### Step 7: Analyzing racial bias in word vectors asscoiated with professions

Similar to Step 4, we project occpurations onto the racial dimension.

In [15]:
# profession analysis racial
sp = sorted([(E.v(w).dot(v_racial), w) for w in profession_words])

sp[0:20], sp[-20:]

([(-0.19739236, 'parliamentarian'),
  (-0.17668717, 'doctor'),
  (-0.17474556, 'assistant_professor'),
  (-0.15524243, 'aide'),
  (-0.15354116, 'comic'),
  (-0.15234408, 'banker'),
  (-0.15101928, 'painter'),
  (-0.14880778, 'warden'),
  (-0.13810316, 'prosecutor'),
  (-0.13234167, 'cinematographer'),
  (-0.12777938, 'judge'),
  (-0.119778596, 'politician'),
  (-0.117808595, 'baker'),
  (-0.11612019, 'chemist'),
  (-0.11549363, 'director'),
  (-0.10975158, 'footballer'),
  (-0.10933608, 'receptionist'),
  (-0.105675615, 'vice_chancellor'),
  (-0.10368978, 'evangelist'),
  (-0.10231235, 'poet')],
 [(0.08507743, 'plastic_surgeon'),
  (0.0856626, 'missionary'),
  (0.08641774, 'doctoral_student'),
  (0.08855405, 'naturalist'),
  (0.090043336, 'laborer'),
  (0.09558587, 'pharmacist'),
  (0.09658609, 'historian'),
  (0.09718482, 'novelist'),
  (0.099601604, 'paralegal'),
  (0.107786536, 'patrolman'),
  (0.10910435, 'boss'),
  (0.10940388, 'administrator'),
  (0.11292209, 'interior_designer')

## Exercise

Repeat Step 2-4 with debiased word embedding. 

You can use debiaswe debias function to do the debiasing with word sets of your choosing

You can leave equalize_pairs and gender_specific_words blank when coming up with your own groups. We give an example for the case of gender below for you to warm up.

In [16]:
from debiaswe.debias import debias

In [17]:
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f)
print("definitional", defs)

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f)

with open('./data/gender_specific_seed.json', "r") as f:
    gender_specific_words = json.load(f)
print("gender specific", len(gender_specific_words), gender_specific_words[:10])

definitional [['woman', 'man'], ['girl', 'boy'], ['she', 'he'], ['mother', 'father'], ['daughter', 'son'], ['gal', 'guy'], ['female', 'male'], ['her', 'his'], ['herself', 'himself'], ['Mary', 'John']]
gender specific 218 ['actress', 'actresses', 'aunt', 'aunts', 'bachelor', 'ballerina', 'barbershop', 'baritone', 'beard', 'beards']


In [18]:
gender_specific_words

['actress',
 'actresses',
 'aunt',
 'aunts',
 'bachelor',
 'ballerina',
 'barbershop',
 'baritone',
 'beard',
 'beards',
 'beau',
 'bloke',
 'blokes',
 'boy',
 'boyfriend',
 'boyfriends',
 'boyhood',
 'boys',
 'brethren',
 'bride',
 'brides',
 'brother',
 'brotherhood',
 'brothers',
 'bull',
 'bulls',
 'businessman',
 'businessmen',
 'businesswoman',
 'chairman',
 'chairwoman',
 'chap',
 'colt',
 'colts',
 'congressman',
 'congresswoman',
 'convent',
 'councilman',
 'councilmen',
 'councilwoman',
 'countryman',
 'countrymen',
 'czar',
 'dad',
 'daddy',
 'dads',
 'daughter',
 'daughters',
 'deer',
 'diva',
 'dowry',
 'dude',
 'dudes',
 'elder_brother',
 'eldest_son',
 'estranged_husband',
 'estranged_wife',
 'estrogen',
 'ex_boyfriend',
 'ex_girlfriend',
 'father',
 'fathered',
 'fatherhood',
 'fathers',
 'fella',
 'fellas',
 'female',
 'females',
 'feminism',
 'fiance',
 'fiancee',
 'fillies',
 'filly',
 'fraternal',
 'fraternities',
 'fraternity',
 'gal',
 'gals',
 'gelding',
 'gentle

In [19]:
defs

[['woman', 'man'],
 ['girl', 'boy'],
 ['she', 'he'],
 ['mother', 'father'],
 ['daughter', 'son'],
 ['gal', 'guy'],
 ['female', 'male'],
 ['her', 'his'],
 ['herself', 'himself'],
 ['Mary', 'John']]

In [20]:
equalize_pairs

[['monastery', 'convent'],
 ['spokesman', 'spokeswoman'],
 ['Catholic_priest', 'nun'],
 ['Dad', 'Mom'],
 ['Men', 'Women'],
 ['councilman', 'councilwoman'],
 ['grandpa', 'grandma'],
 ['grandsons', 'granddaughters'],
 ['prostate_cancer', 'ovarian_cancer'],
 ['testosterone', 'estrogen'],
 ['uncle', 'aunt'],
 ['wives', 'husbands'],
 ['Father', 'Mother'],
 ['Grandpa', 'Grandma'],
 ['He', 'She'],
 ['boy', 'girl'],
 ['boys', 'girls'],
 ['brother', 'sister'],
 ['brothers', 'sisters'],
 ['businessman', 'businesswoman'],
 ['chairman', 'chairwoman'],
 ['colt', 'filly'],
 ['congressman', 'congresswoman'],
 ['dad', 'mom'],
 ['dads', 'moms'],
 ['dudes', 'gals'],
 ['ex_girlfriend', 'ex_boyfriend'],
 ['father', 'mother'],
 ['fatherhood', 'motherhood'],
 ['fathers', 'mothers'],
 ['fella', 'granny'],
 ['fraternity', 'sorority'],
 ['gelding', 'mare'],
 ['gentleman', 'lady'],
 ['gentlemen', 'ladies'],
 ['grandfather', 'grandmother'],
 ['grandson', 'granddaughter'],
 ['he', 'she'],
 ['himself', 'herself'],

TypeError: debias() missing 1 required positional argument: 'equalize'

In [21]:
debias(E, gender_specific_words, defs, equalize_pairs)

26423 words of dimension 300 : in, for, that, is, ..., Jay, Leroy, Brad, Jermaine
{('PROSTATE_CANCER', 'OVARIAN_CANCER'), ('FELLA', 'GRANNY'), ('PRINCE', 'PRINCESS'), ('Males', 'Females'), ('SON', 'DAUGHTER'), ('CHAIRMAN', 'CHAIRWOMAN'), ('WIVES', 'HUSBANDS'), ('grandson', 'granddaughter'), ('GRANDFATHER', 'GRANDMOTHER'), ('TWIN_BROTHER', 'TWIN_SISTER'), ('gentlemen', 'ladies'), ('He', 'She'), ('SCHOOLBOY', 'SCHOOLGIRL'), ('son', 'daughter'), ('Male', 'Female'), ('spokesman', 'spokeswoman'), ('FATHERS', 'MOTHERS'), ('GRANDPA', 'GRANDMA'), ('kings', 'queens'), ('Father', 'Mother'), ('boys', 'girls'), ('grandpa', 'grandma'), ('Boy', 'Girl'), ('Gentleman', 'Lady'), ('Kings', 'Queens'), ('man', 'woman'), ('fatherhood', 'motherhood'), ('Colt', 'Filly'), ('king', 'queen'), ('Chairman', 'Chairwoman'), ('KINGS', 'QUEENS'), ('FRATERNITY', 'SORORITY'), ('Nephew', 'Niece'), ('MONASTERY', 'CONVENT'), ('Fella', 'Granny'), ('KING', 'QUEEN'), ('Schoolboy', 'Schoolgirl'), ('congressman', 'congresswoma

In [22]:
# profession analysis gender
sp_debiased = sorted([(E.v(w).dot(v_gender), w) for w in profession_words])

sp_debiased[0:20], sp_debiased[-20:]

([(-0.4196325, 'congressman'),
  (-0.4067585, 'businessman'),
  (-0.32398775, 'councilman'),
  (-0.30967075, 'dad'),
  (-0.21665451, 'statesman'),
  (-0.11345412, 'salesman'),
  (-0.073004864, 'monk'),
  (-0.072163954, 'handyman'),
  (-0.04946825, 'minister'),
  (-0.043583874, 'archbishop'),
  (-0.040207226, 'bishop'),
  (-0.03833246, 'commissioner'),
  (-0.03572439, 'surgeon'),
  (-0.033134006, 'trader'),
  (-0.032377213, 'observer'),
  (-0.032095864, 'neurosurgeon'),
  (-0.031450115, 'priest'),
  (-0.031133952, 'skipper'),
  (-0.029659169, 'lawmaker'),
  (-0.029511213, 'commander')],
 [(0.029965645, 'teenager'),
  (0.030237027, 'instructor'),
  (0.03094614, 'student'),
  (0.031116948, 'paralegal'),
  (0.032039408, 'bookkeeper'),
  (0.032434624, 'cinematographer'),
  (0.034329075, 'graphic_designer'),
  (0.034705658, 'lifeguard'),
  (0.03566652, 'janitor'),
  (0.03597192, 'drummer'),
  (0.042120155, 'wrestler'),
  (0.043902237, 'hairdresser'),
  (0.04813315, 'firefighter'),
  (0.23776

In [23]:
# analogies gender
a_gender_debiased = E.best_analogies_dist_thresh(v_gender)

for (a,b,c) in a_gender_debiased:
    print(a+"-"+b)

Computing neighbors
Mean: 10.218597434053665
Median: 7.0
spokeswoman-spokesman
filly-colt
mothers-fathers
princess-prince
women-men
girls-boys
grandmother-grandfather
granddaughters-grandsons
estrogen-testosterone
queens-kings
daughter-son
daughters-sons
woman-man
queen-king
chairwoman-chairman
moms-dads
girl-boy
niece-nephew
motherhood-fatherhood
congresswoman-congressman
sisters-brothers
females-males
councilwoman-councilman
she-he
ladies-gentlemen
herself-himself
her-his
aunt-uncle
schoolgirl-schoolboy
businesswoman-businessman
sorority-fraternity
twin_sister-twin_brother
female-male
ovarian_cancer-prostate_cancer
grandma-grandpa
ex_boyfriend-ex_girlfriend
husbands-wives
mare-gelding
mom-dad
convent-monastery
mother-father
sister-brother
gals-dudes
granddaughter-grandson
actress-actor
lesbian-gay
compatriot-countryman
husband-younger_brother
gal-dude
hers-theirs
heroine-protagonist
feminism-feminist
actresses-actors
childhood-boyhood
waitress-waiter
kid-guy
me-him
mommy-daddy
aunts-